In [22]:
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

Using TensorFlow backend.


In [35]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from PIL import Image

from glob import glob

In [24]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'Dogs_4/train'
validation_data_dir = 'Dogs_4/validation'

In [25]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 38140 images belonging to 2 classes.
Found 13094 images belonging to 2 classes.


In [26]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [5]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(Flatten())
model_vgg.add(Dense(64))
model_vgg.add(Activation('relu'))
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(1))
model_vgg.add(Activation('sigmoid'))

In [27]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [28]:
nb_epoch = 30
nb_train_samples = 38140
nb_validation_samples = 13094

In [29]:
filepath="rajDogs4-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [30]:
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples//16, #40661 // batch_size
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples//16, #15502//batch_size
        callbacks=callbacks_list)

Epoch 1/30
148/148 [==============================] - 341s - loss: 0.7381 - acc: 0.4996 - val_loss: 1.0647 - val_acc: 0.4998
Epoch 2/30
148/148 [==============================] - 341s - loss: 0.6953 - acc: 0.5359 - val_loss: 0.6793 - val_acc: 0.5745
Epoch 3/30
148/148 [==============================] - 341s - loss: 0.7021 - acc: 0.5215 - val_loss: 0.6885 - val_acc: 0.5411
Epoch 4/30
148/148 [==============================] - 341s - loss: 0.6948 - acc: 0.5232 - val_loss: 0.6835 - val_acc: 0.5675
Epoch 5/30
148/148 [==============================] - 341s - loss: 0.6880 - acc: 0.5507 - val_loss: 0.6801 - val_acc: 0.5843
Epoch 6/30
148/148 [==============================] - 342s - loss: 0.6822 - acc: 0.5773 - val_loss: 0.6737 - val_acc: 0.5725
Epoch 7/30
148/148 [==============================] - 342s - loss: 0.6802 - acc: 0.5688 - val_loss: 0.6811 - val_acc: 0.5651
Epoch 8/30
148/148 [==============================] - 342s - loss: 0.6822 - acc: 0.5815 - val_loss: 0.6654 - val_acc: 0.6032


In [31]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.65053625054754949, 0.62390237603569476]

In [32]:
nb_validation_samples/32

409

In [33]:
val_predictions = model.predict_generator(validation_generator,410)

In [36]:
preds = pd.DataFrame(val_predictions)
files = pd.DataFrame(validation_generator.filenames)
dogs4_predictions = files.join(preds,lsuffix=0, rsuffix= 11594)
dogs4_predictions.columns = ['path','probability']
dogs4_predictions.head()

,path,probability
0,bad/a0828a1a4c51a7d34d3f2a21838389899659790b.jpg,0.632456
1,bad/091457a94f16171448dfb8a62f73ee17c6d404b1.jpg,0.385794
2,bad/449c769c320b32999e741814e603e4ba800db7e0.jpg,0.372878
3,bad/8906be51c0700528ef5d8b6acf41bf5a2fbd3ca4.jpg,0.395957
4,bad/3850b6fb6ca17bbead06b62b8dfe94eac65eeece.jpg,0.426769


In [37]:
dogs4_predictions.to_csv('dogs4_predictions.csv')